<a href="https://colab.research.google.com/github/rtealwitter/naturalexperiments/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# naturalexperiments

The `naturalexperiments` package is a comprehensive toolbox for treatment effect estimation. The package includes a variety of datasets, estimators, and evaluation metrics for treatment effect estimation. The package is designed to be accessible to researchers and practitioners who are new to treatment effect estimation and to provide a comprehensive set of tools for experienced researchers.

For faster (but still not fast) evaluation, we suggest selecting a GPU runtime if using Google Colab.

In [ ]:
!pip install naturalexperiments

In [2]:
import naturalexperiments as ne

## Datasets

We introduce a novel treatment effect dataset from an early childhood literacy natural experiment. The treatment in the experiment is participation in Reach Out and Read Colorado (RORCO). The dataset has an observational version called RORCO Real with real literacy outcomes and a semi-synthetic version called RORCO for estimator evaluation purposes.

In addition to RORCO and RORCO Real, we provide easy access to standard treatment effect datasets including ACIC 2016, ACIC 2017, IHDP, Jobs, News, and Twins.

All of the datasets can be loaded using the `dataloaders` object.

In [15]:
dataset = 'RORCO'

X, y, z = ne.dataloaders[dataset]()

## Estimators

We propose a novel, theoretically motivated doubly robust estimator called Double-Double. In addition to Double-Double, we provide implementations of more than 20 established estimators from the literature.

All of the estimators can be easily loaded using the `methods` object.

In [4]:
method = 'Double-Double'
estimator = ne.methods[method]

We can check the other estimators by inspecting the keys of `methods`.

In [22]:
print(list(ne.methods.keys()))

['Regression Discontinuity', 'Propensity Stratification', 'Direct Difference', 'Adjusted Direct', 'Horvitz-Thompson', 'TMLE', 'Off-policy', 'Double-Double', 'Doubly Robust', 'Direct Prediction', 'SNet', 'FlexTENet', 'OffsetNet', 'TNet', 'TARNet', 'DragonNet', 'SNet3', 'DRNet', 'RANet', 'PWNet', 'RNet', 'XNet']


Each method takes the following arguments: the covariates `X`, the outcomes `y`, the treatment assignment `z`, propensity score estimates `p`, and a function for training `train` predictions in the estimator.

We can use the `estimate_propensity` function to estimate the propensity scores.

In [17]:
p = ne.estimate_propensity(X, z) # Roughly 1 minute

Then, with the propensity scores, we can estimate the treatment effects.

In [18]:
estimated_effect = estimator(X, y, z, p, ne.train) # Roughly 1 minute

In [19]:
true_effect = (y['y1'] - y['y0']).mean()
print(f'The treatment effect is {true_effect} while the treatment effect estimated by {method} is {estimated_effect}.')

The treatment effect is 0.08955221295798275 while the treatment effect estimated by Double-Double is 0.08833388528237851.


By default, `train` trains a three-layer neural network with 100 units in each layer and ReLU activations.
Some estimators, such as regression discontinuity, do not use the training functions and some estimators, such as the CATENet estimators, use custom training functions defined in the estimator.

## Exploring the Datasets

We can explore the datasets with a tabular comparison and several figures.

In [8]:
# Produces a markdown table comparing the size, number of variables, treatment rate, etc.
ne.dataset_table(ne.dataloaders, print_md=True)

Dataset       Size    Variables    % Treated    Cross Entropy    Corr(y1, p)    Corr(y0, p)
----------  ------  -----------  -----------  ---------------  -------------  -------------
ACIC 2016     4802           54         18.4           0.379         0.129         0.0393
ACIC 2017     4302           50         51             0.606        -0.116        -0.0247
IHDP           747           26         18.6           0.459         0.0785        0.0199
JOBS           722            8         41.1           0.0827        0.0356        0.0766
NEWS          5000            3         45.8           0.545         0.858        -0.563
TWINS        50820           40         49             0.506         0.00133      -0.000665
RORCO Real    4178           78         25.3           0.162        -0.027        -0.101
RORCO        21663           78         49.4           0.151        -0.979        -0.98


## Benchmarking the Estimators

We can benchmark the estimators on the datasets using the `compute_variance` function.

In [20]:
methods = {name: ne.methods[name] for name in ['Double-Double', 'Regression Discontinuity', 'DragonNet']}

variance, times = ne.compute_variance(methods, dataset, num_runs=5) # Roughly 1 minute per run

100%|██████████| 5/5 [03:50<00:00, 46.03s/it]


Due to the computational complexity of some estimators (e..g, the CATENets), the benchmark subsamples the data by default. We can adjust the subsample size with the `limit` argument. Even then, many estimators may take a long time to run.

Once we benchmark the estimators, we can print the results in a table.

In [21]:
ne.benchmark_table(variance, times, print_md=True)

| Method                   |     Mean |   1st Quartile |   2nd Quartile |   3rd Quartile |   Time (s) |
|--------------------------|----------|----------------|----------------|----------------|------------|
| Double-Double            | 1.04e-05 |       8.18e-06 |       1.01e-05 |       1.38e-05 |   27.6     |
| Regression Discontinuity | 0.00467  |       0.00411  |       0.00493  |       0.00588  |    0.00115 |
| TARNet                   | 2.72e-05 |       2.72e-05 |       2.72e-05 |       2.72e-05 |   99.2     |
| DragonNet                | 0.0223   |       0.0217   |       0.0227   |       0.0252   |    4.62    |


## Additional Features

The `naturalexperiments` package includes additional features for comprehensively evaluating treatment effect estimation.

There are functions for computing the empirical variance as a function of the sample size, the correlation in the outcomes, and the propensity score accuracy.

These functions and more appear in the `paper_experiments` folder (as the name suggests, the folder includes code to reproduce the results in the paper). Because some experiments are computationally intensive, the functions are designed to run in parallel by writing the results to a shared cache.